In [1]:
import pandas as pd
import os
os.getcwd()

'c:\\Users\\T-Gamer\\Documents\\SideDrive\\UFMA\\2022.1\\Topicos Especiais (NLP)\\Exercicios\\Trabalho Final\\Implementação\\source'

In [2]:
dataset_folder = "../resources/datasets/StanfordSentimentTreebank"
os.chdir(dataset_folder)

In [3]:
def correct_accented_chars(text):
    pairs_to_correct = [("Ã¡",    "á"),
                        ("Ã\xa0", "à"),
                        ("Ã¢",    "â"),
                        ("Ã£",    "ã"),
                        ("Ã©",    "é"),
                        ("Ã¨",    "è"),
                        ("Ã\xad", "í"),
                        ("Ã¯",    "ï"),
                        ("Ã³",    "ó"),
                        ("Ã´",    "ô"),
                        ("Ã¶",    "ö"),
                        ("Ã¼",    "ü"),
                        ("Ã»",    "û"),
                        ("Ã¦",    "æ"),
                        ("Ã±",    "ñ"),
                        ("Ã§",    "ç"),
                        ("Â\xa01\\", "\xa01\\"),
                        ("-LRB-", "("),
                        ("-RRB-", ")")]
    for bad_accent_code, correct_char in pairs_to_correct :
        text = text.replace(bad_accent_code, correct_char)
    return text

In [4]:
sentence_id = []
text_column = []
with open("./datasetSentences.txt", "r", encoding='utf-8') as f :
    for line in f.readlines()[1 : ] :
        id = line.split()[0]
        text = line[len((id)) : ].strip()
        sentence_id.append(id)
        text_column.append(correct_accented_chars(text))

In [5]:
dictionary = {}
with open("./dictionary.txt", "r", encoding='utf-8') as f :
    for line in f.readlines() :
        splitpoint = line.rindex("|")
        phrase    = line[ : splitpoint].strip()
        phrase_id = int(line[splitpoint + 1 : ].strip())
        dictionary[phrase] = phrase_id

In [6]:
phrase_id_column = []
for sentence_text in text_column :
    phrase_id_column.append(dictionary[sentence_text])

In [7]:
text_column      = pd.Series(text_column,      sentence_id, name="text"     )
phrase_id_column = pd.Series(phrase_id_column, sentence_id, name="phrase_id")

dataset = pd.concat([text_column, phrase_id_column], axis=1)
dataset

,text,phrase_id
1,The Rock is destined to be the 21st Century 's...,226166
2,The gorgeously elaborate continuation of `` Th...,226300
3,Effective but too-tepid biopic,13995
4,If you sometimes like to go to the movies to h...,14123
5,"Emerges as something rare , an issue movie tha...",13999
...,...,...
11851,A real snooze .,222071
11852,No surprises .,225165
11853,We 've seen the hippie-turned-yuppie plot befo...,226985
11854,Her fans walked out muttering words like `` ho...,223632


In [23]:
splitset_column = []
sentence_id_SPLIT = []
with open("./datasetSplit.txt", "r") as f :
    for line in f.readlines()[1 : ] :
        line = line.split(",")
        sentence_id_SPLIT.append(line[0])
        splitset_column.append(int(line[1]))


In [24]:
splitset_column = pd.Series(splitset_column, sentence_id_SPLIT, name="splitset")
splitset_column

1        1
2        1
3        2
4        2
5        2
        ..
11851    1
11852    1
11853    1
11854    1
11855    1
Name: splitset, Length: 11855, dtype: int64

In [27]:
dataset = pd.concat([dataset, splitset_column], axis=1)
dataset

,text,phrase_id,splitset
1,The Rock is destined to be the 21st Century 's...,226166,1
2,The gorgeously elaborate continuation of `` Th...,226300,1
3,Effective but too-tepid biopic,13995,2
4,If you sometimes like to go to the movies to h...,14123,2
5,"Emerges as something rare , an issue movie tha...",13999,2
...,...,...,...
11851,A real snooze .,222071,1
11852,No surprises .,225165,1
11853,We 've seen the hippie-turned-yuppie plot befo...,226985,1
11854,Her fans walked out muttering words like `` ho...,223632,1


In [28]:
sentiment_labels = {}
with open("./sentiment_labels.txt", "r") as f :
    for line in f.readlines()[1 : ] :
        line = line.split("|")
        sentiment_labels[int(line[0].strip())] = float(line[1].strip())

In [29]:
class_column = []
for phrase_id in phrase_id_column :
    class_column.append(sentiment_labels[phrase_id])

In [31]:
class_column = pd.Series(class_column, sentence_id, name="class")

dataset = pd.concat([dataset, class_column], axis=1)
dataset

,text,phrase_id,splitset,class
1,The Rock is destined to be the 21st Century 's...,226166,1,0.69444
2,The gorgeously elaborate continuation of `` Th...,226300,1,0.83333
3,Effective but too-tepid biopic,13995,2,0.51389
4,If you sometimes like to go to the movies to h...,14123,2,0.73611
5,"Emerges as something rare , an issue movie tha...",13999,2,0.86111
...,...,...,...,...
11851,A real snooze .,222071,1,0.11111
11852,No surprises .,225165,1,0.22222
11853,We 've seen the hippie-turned-yuppie plot befo...,226985,1,0.75000
11854,Her fans walked out muttering words like `` ho...,223632,1,0.13889


In [32]:
dataset.to_csv("SST2.csv")